In [1]:
from infomap import Infomap
from notebook_utils import setup
import pandas as pd
import networkx as nx
from collections import defaultdict

setup()

In [2]:
DATE = "16-dec"
DATA_DIR = "../data/{}/".format(DATE)
EXPORT_DIR = "../data/dataframes/{}/".format(DATE)

In [3]:
df_retweets = pd.read_pickle(EXPORT_DIR + 'df_retweets.pickle')

In [86]:
df_users.shape[0]

1388621

In [3]:
df_users = pd.read_pickle(EXPORT_DIR + 'df_users.pickle')

In [4]:
def get_node_id(user_id, node_ids):
    if (user_id not in node_ids):
        node_counter = len(node_ids)
        node_ids[user_id] = node_counter

    return node_ids[user_id]

def build_graph(N = 1000000):
    print("Building graph...")
    graph = nx.DiGraph()

    unknown_users = set()
    node_counter = 0
    node_id_map = {}
    
    def add_user_to_graph(graph, user_id):
        node_id = get_node_id(user_id, node_id_map)
        if (not graph.has_node(node_id)):
            if (user_id in df_users.index):
                graph.add_node(
                    node_id,
                    user_id=user_id,
                    label=df_users.at[user_id, "handle"],
                    followers=df_users.at[user_id, "followers_count"]
                )
            else:
                unknown_users.add(user_id)
                graph.add_node(
                    node_id,
                    user_id=user_id
                )
        return node_id
    
    def add_retweet_to_graph(graph, retweet_author, retweet_user):
        source = add_user_to_graph(graph, retweet_author)
        target = add_user_to_graph(graph, retweet_user)
        
        existing_edge_data = graph.get_edge_data(source, target)
        if (existing_edge_data):
            existing_edge_data["weight"] += 1
        else: 
            graph.add_edge(source, target, weight=1)
    
    retweet_rows = zip(df_retweets["retweetedFrom_user"][:N], df_retweets["user"][:N])
    i = 0
    for retweet_author, retweet_user in retweet_rows:
        add_retweet_to_graph(graph, retweet_author, retweet_user)
        
        if (N > 200000 and i % 100000 == 0):
            print("{}/{}".format(i, N))
        i += 1

    print("Done building graph, {} nodes (users) and {} edges (retweets)".format(graph.number_of_nodes(), graph.number_of_edges()))
    print("Unknown users: {}".format(len(unknown_users)))
    return graph, node_id_map

In [5]:
graph, node_id_map = build_graph(10000000000)

Building graph...
0/10000000000
100000/10000000000
200000/10000000000
300000/10000000000
400000/10000000000
500000/10000000000
600000/10000000000
700000/10000000000
800000/10000000000
900000/10000000000
1000000/10000000000
1100000/10000000000
1200000/10000000000
1300000/10000000000
1400000/10000000000
1500000/10000000000
1600000/10000000000
1700000/10000000000
1800000/10000000000
1900000/10000000000
2000000/10000000000
2100000/10000000000
2200000/10000000000
2300000/10000000000
2400000/10000000000
2500000/10000000000
2600000/10000000000
2700000/10000000000
2800000/10000000000
2900000/10000000000
3000000/10000000000
3100000/10000000000
3200000/10000000000
3300000/10000000000
3400000/10000000000
3500000/10000000000
3600000/10000000000
3700000/10000000000
3800000/10000000000
3900000/10000000000
4000000/10000000000
4100000/10000000000
4200000/10000000000
4300000/10000000000
4400000/10000000000
4500000/10000000000
4600000/10000000000
4700000/10000000000
4800000/10000000000
4900000/100000000

In [19]:
def community_distribution(communities):
    return [(cluster_id, len(c)) for cluster_id, c in enumerate(communities)]

def get_largest_subgraph(graph):
    connected_components = [graph.subgraph(c) for c in nx.connected_components(graph)]
    return max(connected_components, key=len)

In [26]:
def ordered_community_maps (communities):
    # Order clusters by size
    node_community_map = {}
    community_node_map = defaultdict(lambda: set())
    ordered_communities = sorted(communities, key=len, reverse=True)
    for label, cluster in enumerate(ordered_communities):
        cluster_size = len(cluster)
        for node in cluster:
            node_community_map[node] = label
            community_node_map[label].add(node)
    return node_community_map, dict(community_node_map)

def find_communities_Infomap(G, infomap_options="", weighted=True):
    """
    Partition network with the Infomap algorithm.
    Annotates nodes with 'community' id and return number of communities found.
    """
    infomapX = Infomap(infomap_options + " --seed 42")

    print("Building Infomap network from a NetworkX graph...")
    print("Options: " + infomap_options)
    for source, target, data in G.edges(data=True):
        if (weighted):
            infomapX.network.addLink(source, target, data["weight"])
        else:
            infomapX.network.addLink(source, target)

    print("Find communities with Infomap...")
    infomapX.run()

    print("Found {} modules with codelength: {}".format(infomapX.numTopModules(), infomapX.codelength))

    community_node_map = defaultdict(lambda: set())
    for node in infomapX.iterLeafNodes():
        community_id = node.moduleIndex()
        community_node_map[community_id].add(node.physicalId)

    return ordered_community_maps(community_node_map.values())

In [8]:
clustering = {}

In [21]:
subgraph = cached_graph["graph"].subgraph(list(cached_graph["graph"].nodes())[:500])

In [22]:
clustering["default_params"] = find_communities_Infomap(cached_graph["graph"], "", weighted=False)
node_community_map, community_map = clustering["default_params"]

print("Top 5 communities:")
community_distribution(community_map.values())[:5]

Building Infomap network from a NetworkX graph...
Options: 
Find communities with Infomap...
Found 4 modules with codelength: 4.440074876041761
Cluster 0
56
Cluster 1
11
Cluster 2
6
Cluster 3
5
Top 5 communities:


[(0, 56), (1, 11), (2, 6), (3, 5)]

In [40]:
cached_graph["clustering_undirected_unweighted"] = find_communities_Infomap(cached_graph["graph"], "", weighted=False)
node_community_map, communities = cached_graph["clustering_undirected_unweighted"]
print("Top 5 communities:")
community_distribution(cached_graph["clustering_undirected_unweighted"][1].values)[:5]

Building Infomap network from a NetworkX graph...
Options: 


KeyboardInterrupt: 

In [42]:
print("Top 5 communities:")
community_distribution(cached_graph["clustering_undirected_unweighted"][1].values())[:5]

Top 5 communities:


[(0, 965198), (1, 811421), (2, 20917), (3, 7482), (4, 6958)]

In [25]:
cached_graph["clustering_directed_unweighted"] = find_communities_Infomap(cached_graph["graph"], "--directed", weighted=False)

Cluster 41886
1
Cluster 41887
1
Cluster 41888
1
Cluster 41889
1
Cluster 41890
1
Cluster 41891
1
Cluster 41892
1
Cluster 41893
1
Cluster 41894
1
Cluster 41895
1
Cluster 41896
1
Cluster 41897
1
Cluster 41898
1
Cluster 41899
1
Cluster 41900
1
Cluster 41901
1
Cluster 41902
1
Cluster 41903
1
Cluster 41904
1
Cluster 41905
1
Cluster 41906
1
Cluster 41907
1
Cluster 41908
1
Cluster 41909
1
Cluster 41910
1
Cluster 41911
1
Cluster 41912
1
Cluster 41913
1
Cluster 41914
1
Cluster 41915
1
Cluster 41916
1
Cluster 41917
1
Cluster 41918
1
Cluster 41919
1
Cluster 41920
1
Cluster 41921
1
Cluster 41922
1
Cluster 41923
1
Cluster 41924
1
Cluster 41925
1
Cluster 41926
1
Cluster 41927
1
Cluster 41928
1
Cluster 41929
1
Cluster 41930
1
Cluster 41931
1
Cluster 41932
1
Cluster 41933
1
Cluster 41934
1
Cluster 41935
1
Cluster 41936
1
Cluster 41937
1
Cluster 41938
1
Cluster 41939
1
Cluster 41940
1
Cluster 41941
1
Cluster 41942
1
Cluster 41943
1
Cluster 41944
1
Cluster 41945
1
Cluster 41946
1
Cluster 41947
1
Cluster 

In [27]:
print("Top 5 communities:")
community_distribution(cached_graph["clustering_directed_unweighted"][1].values)[:5]

Top 5 communities:


[(0, 860622), (1, 455428), (2, 329262), (3, 33589), (4, 24205)]

In [36]:
print("Top 5 communities:")
community_distribution(cached_graph["clustering_directed_weighted"][1].values())[:5]

Top 5 communities:


[(0, 860976), (1, 437783), (2, 342184), (3, 33587), (4, 23414)]

In [41]:
community_node_map = defaultdict(lambda: set())

for n, c in node_community_map.items():
    community_node_map[c].add(n)

In [44]:
for c, nodes in list(community_node_map.items())[:10]:
    print(len(nodes))

342184
860976
437783
14893
33587
23414
13767
10728
13202
2841


In [10]:
clustering["directed"] = find_communities_Infomap(graph, "--directed", weighted=True)
node_community_map, communities = clustering["directed"]

print("Top 5 communities:")
community_distribution(communities)[:5]

Building Infomap network from a NetworkX graph...
Options: --directed
Find communities with Infomap...
Found 51605 modules with codelength: 14.213045797863082
Top 5 communities:


[860976, 437783, 342184, 33587, 23414]

In [10]:
def cluster_users(df_users, node_id_map, node_community_map):
    clusters = []
    missing_in_graph = 0
    missing_assignment = 0
    for user_id, user in df_users.iterrows():
        if user_id in node_id_map:
            node_id = node_id_map[user_id]
            if node_id in node_community_map:
                cluster = node_community_map[node_id]
                clusters.append(cluster)
            else:
                missing_assignment += 1
                clusters.append(-1)    
        else:
            missing_in_graph += 1
            clusters.append(-1)
    print("{} missing assignment".format(missing_assignment))
    print("{} missing in graph".format(missing_in_graph))
    return clusters

In [75]:
all_user_clusters = cluster_users(df_users)

2122 missing assignment
671282 missing in graph


In [2]:
import pickle 
with open("./cached_graph.pickle", "rb") as f:
    cached_graph = pickle.load(f)

In [46]:
with open("./cached_graph.pickle", "wb") as f:
    pickle.dump(cached_graph, f)

In [89]:
import pickle
cached_graph = {
    "all_user_clusters": all_user_clusters,
    "graph": graph,
    "node_id_map": node_id_map,
    "clustering_directed": clustering["directed"]
}

with open("./cached_graph.pickle", "wb") as f:
    pickle.dump(cached_graph, f)

In [7]:
df_users["cluster"] = cached_graph["all_user_clusters"]

In [8]:
df_users_by_cluster = df_users.sort_values(["cluster", "followers_count"], ascending=False).set_index("cluster")

In [35]:
cached_graph.keys()

dict_keys(['all_user_clusters', 'graph', 'node_id_map', 'clustering_directed'])

In [63]:
def print_cluster_stats(users_by_cluster, N=3, top_users=5):
    for c in range(N): 
        print("Cluster {}".format(c))
        cluster_users = users_by_cluster[users_by_cluster.index == c]
        print("Number of nodes in cluster:", len(cached_graph["clustering_directed"][1][c]))
        print("Number of identified users:", cluster_users.shape[0])
        print("Top users")
        print(cluster_users[["handle", "followers_count"]][:top_users])
        #print("Number of users", cluster_users.shape[0])
        print("- mean {:,.1f}".format(cluster_users["followers_count"].mean()))
        print("- min {:,}".format(cluster_users["followers_count"].min()))
        print("- max {:,}".format(cluster_users["followers_count"].max()))
        print("- median {:,.0f}".format(cluster_users["followers_count"].median()))
        print()

In [56]:
# Top users
df_weekly_top_users = df_users = pd.read_pickle(EXPORT_DIR + 'df_weekly_top_users.pickle')
df_top_users = df_weekly_top_users.groupby(["user", "handle", "name", "followers_count", "verified"]).sum().reset_index().set_index("user")
df_top_users["cluster"] = cluster_users(df_top_users, cached_graph["node_id_map"], cached_graph["clustering_directed"][0])
top_users_by_cluster = df_top_users.sort_values(["cluster", "followers_count"], ascending=[True, False]).set_index("cluster")

0 missing assignment
21 missing in graph


In [64]:
print_cluster_stats(top_users_by_cluster, 5, 10)

Cluster 0
Number of nodes in cluster: 860976
Number of identified users: 253
Top users
                 handle  followers_count
cluster                                 
0           no_silenced           105830
0        HelloTeamTrump            95669
0           DoniTheDon_            56112
0            RedPillMaC            43937
0              USSANews            40389
0             vision835            39233
0               WvTrump            33874
0                tan123            32339
0            JJPPATRIOT            30319
0         ConcernedHigh            29927
- mean 4,119.5
- min 0
- max 105,830
- median 708

Cluster 1
Number of nodes in cluster: 437783
Number of identified users: 69
Top users
                 handle  followers_count
cluster                                 
1               nytimes         47663855
1        HillaryClinton         29737638
1               Reuters         22518102
1                    AP         14507714
1         BernieSanders         132380

In [65]:
print_cluster_stats(df_users_by_cluster, 5, 10)

Cluster 0
Number of nodes in cluster: 860976
Number of identified users: 247484
Top users
                handle  followers_count
cluster                                
0            wikileaks          5611146
0           Jim_Jordan          1955593
0        foxandfriends          1363214
0            F1abraham          1127886
0           TobyTurner           973688
0          behindwoods           959045
0         oliverpocher           757462
0          ewnreporter           695846
0          futureshift           654047
0             PAMsLOvE           645323
- mean 2,272.8
- min 0
- max 5,611,146
- median 418

Cluster 1
Number of nodes in cluster: 437783
Number of identified users: 253438
Top users
                 handle  followers_count
cluster                                 
1                cnnbrk         59420316
1                   CNN         50395181
1               nytimes         47663855
1           BBCBreaking         46468749
1        HillaryClinton         29737638


In [271]:
to_keep = [node_id for node_id, degrees in graph.degree() if degrees > 1]
print("Keeping {} nodes".format(len(to_keep)))
graph_filtered_by_degrees = graph.subgraph(to_keep)

(0, 1800)
Keeping 992895 nodes


In [272]:
node_community_map, communities = find_communities_Infomap(graph_filtered_by_degrees, "--directed")

print("Top 5 communities:")
community_distribution(communities)[:5]

Building Infomap network from a NetworkX graph...
Options: --directed
Find communities with Infomap...
Found 36633 modules with codelength: 14.24360555160141
Top 5 communities:


[362262, 303625, 207425, 21753, 14978]

In [251]:
node_community_map[node_id_map["25073877"]]
len(communities[:2][1])

437783

In [220]:
node_community_map, communities = find_communities_Infomap(graph, "--directed --preferred-number-of-modules 2")

print("Top 5 communities:")
community_distribution(communities)[:5]

Building Infomap network from a NetworkX graph...
Find communities with Infomap...
Found 5118 modules with codelength: 13.547634251060105
Top 5 communities:


[191403, 81, 30, 26, 18]

In [170]:
from cdlib.algorithms import infomap

def find_communities_InfomapCdlib(graph):
    clustering = infomap(graph)
    print("Found {} communities".format(len(clustering.communities)))
    return clustering.to_node_community_map(), clustering.communities

largest_subgraph = get_largest_subgraph(graph.to_undirected())

node_community_map, communities = find_communities_InfomapCdlib(graph.to_undirected())

print("Top 5 communities:")
community_distribution(communities)[:5]

Found 5169 communities
Top 5 communities:


[118352, 63712, 3977, 1833, 638]